# Leg Kinematics

## Dependencies

In [9]:
import sys
sys.path.append('../src')
from leg_kinematics import LegKinematics

from math import *
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Line3DCollection
from IPython.display import HTML

## Verify Forward Kinematics

In [ ]:
legkinematics = LegKinematics()
N = 100  # Number of frames

# Define oscillating angles
angles0 = np.linspace(0, np.pi/2, N//2)
angles0 = np.concatenate([angles0, angles0[::-1]])  # Oscillate
angles1 = np.linspace(np.pi, np.pi/2, N)
angles5 = np.linspace(np.pi/2, 0, N)

# Create figure with two subplots
fig = plt.figure(figsize=(12, 6))

ax1 = fig.add_subplot(121, projection='3d')
ax2 = fig.add_subplot(122, projection='3d')

# Set limits and labels for both subplots
for ax in [ax1, ax2]:
  ax.set_xlim3d([-15, 20])
  ax.set_ylim3d([-30, 5])
  ax.set_zlim3d([-15, 15])
  ax.set_xlabel("X")
  ax.set_ylabel("Y")
  ax.set_zlabel("Z")

# Set different viewpoints
ax1.view_init(vertical_axis='y', azim=90)
ax2.view_init(vertical_axis='y', azim=30)

# Initialize empty lines for both subplots
line1_1, = ax1.plot([], [], [], 'blue', markersize=5)
line1_2, = ax1.plot([], [], [], 'blue', markersize=5)
scatter1 = ax1.scatter([], [], [], color='red', s=50, label='Joints')

line2_1, = ax2.plot([], [], [], 'blue', markersize=5)
line2_2, = ax2.plot([], [], [], 'blue', markersize=5)
scatter2 = ax2.scatter([], [], [], color='red', s=50, label='Joints')

# Animation update function
def update(frame):
  angle0 = angles0[frame]
  angle1 = angles1[frame]
  angle5 = angles5[frame]

  legkinematics.set_motor_angles([angle0, angle1, angle5])
  points = legkinematics.get_points()
  if points is not None:
    # Update first subplot (Side view)
    line1_1.set_data(points[[0, 1, 2, 3], 0], points[[0, 1, 2, 3], 1])
    line1_2.set_data(points[[0, 1, 2, 4, 5], 0], points[[0, 1, 2, 4, 5], 1])
    line1_1.set_3d_properties(points[[0, 1, 2, 3], 2])
    line1_2.set_3d_properties(points[[0, 1, 2, 4, 5], 2])
    scatter1._offsets3d = (points[:, 0], points[:, 1], points[:, 2])

    # Update second subplot (Front view)
    line2_1.set_data(points[[0, 1, 2, 3], 0], points[[0, 1, 2, 3], 1])
    line2_2.set_data(points[[0, 1, 2, 4, 5], 0], points[[0, 1, 2, 4, 5], 1])
    line2_1.set_3d_properties(points[[0, 1, 2, 3], 2])
    line2_2.set_3d_properties(points[[0, 1, 2, 4, 5], 2])
    scatter2._offsets3d = (points[:, 0], points[:, 1], points[:, 2])

  return line1_1, line1_2, scatter1, line2_1, line2_2, scatter2

# Run animation
ani = FuncAnimation(fig, update, frames=N, interval=50)
HTML(ani.to_jshtml())

## Verify Inverse Kinematics

In [ ]:
# Given angles
angle0, angle1, angle5 = np.pi/4, np.pi, np.pi/2 
print("Forward kinematics")
print("Given Angles: ", angle0, angle1, angle5)
legkinematics.set_motor_angles([angle0, angle1, angle5])
ee_point = legkinematics.get_ee_point()
print("Points: ", ee_point)

print("\nInverse kinematics")
print("Given Point: ", ee_point)
legkinematics.set_ee_point(ee_point)
angles = legkinematics.get_angles()
print("Angles: ", angles[0], angles[1], angles[5])

## Verify Differential Forward Kinematics

In [12]:
class Verifier:
  def __init__(self):
    # Create figure with two subplots
    self.fig = plt.figure(figsize=(15, 5))
        
    # Create 3D subplot for mechanism
    self.ax_mech = self.fig.add_subplot(121, projection='3d')
    
    # Set up axes
    self.ax_mech.set_xlabel("X")
    self.ax_mech.set_ylabel("Y")
    self.ax_mech.set_zlabel("Z")
    self.ax_mech.set_xlim(-15, 20)
    self.ax_mech.set_ylim(-30, 5)
    self.ax_mech.set_zlim(-10, 10)
    self.ax_mech.set_box_aspect([35, 35, 20])
    self.ax_mech.grid(True)
    self.ax_mech.set_title('Leg Mechanism')

    # Create 2D subplot for velocity
    self.ax_vel = self.fig.add_subplot(122)
    self.ax_vel.set_xlim(0, 2)
    self.ax_vel.set_ylim(-20, 20)
    self.ax_vel.grid(True)
    self.ax_vel.set_title('P3 Velocity Components')
    self.ax_vel.set_xlabel('Time (s)')
    self.ax_vel.set_ylabel('Velocity')

    # Data storage
    self.times = []
    self.actual_velocities = []
    self.calculated_velocities = []
    self.points_history = []

    # Dummy line for animation update
    self.dummyLine, = self.ax_mech.plot([], [], [])

    # Initialize velocity plot lines
    self.vx_actual, = self.ax_vel.plot([], [], label='Vx Actual')
    self.vy_actual, = self.ax_vel.plot([], [], label='Vy Actual')
    self.vz_actual, = self.ax_vel.plot([], [], label='Vz Actual')
    self.vx_calc, = self.ax_vel.plot([], [], label='Vx Calc')
    self.vy_calc, = self.ax_vel.plot([], [], label='Vy Calc')
    self.vz_calc, = self.ax_vel.plot([], [], label='Vz Calc')
    
    self.ax_mech.view_init(vertical_axis='y', azim=30)
    self.ax_vel.legend(bbox_to_anchor=(1.04, 1), loc="upper left")

  def update(self, frame):
    points = self.points_history[frame]

    self.ax_mech.cla()
    self.ax_mech.set_xlabel("X")
    self.ax_mech.set_ylabel("Y")
    self.ax_mech.set_zlabel("Z")
    self.ax_mech.set_xlim(-15, 20)
    self.ax_mech.set_ylim(-30, 5)
    self.ax_mech.set_zlim(-10, 10)
    self.ax_mech.set_box_aspect([35, 35, 20])
    self.ax_mech.grid(True)
    self.ax_mech.set_title('Leg Mechanism')
    
    # Update chain
    self.ax_mech.plot3D(points[[0, 1, 2, 3], 0], points[[0, 1, 2, 3], 1], points[[0, 1, 2, 3], 2], 'blue')
    self.ax_mech.plot3D(points[[0, 1, 2, 4, 5], 0], points[[0, 1, 2, 4, 5], 1], points[[0, 1, 2, 4, 5], 2], 'blue')
    self.ax_mech.scatter3D(points[:, 0], points[:, 1], points[:, 2], c='r', marker='o')
    
    # Update velocity vector at p3
    if frame > 0:
      actual_vel = self.actual_velocities[frame]
      calc_vel = self.calculated_velocities[frame]

      # Plot velocity vector
      self.ax_mech.quiver(points[3, 0], points[3, 1], points[3, 2], actual_vel[0], actual_vel[1], actual_vel[2], color='g', label='V Actual')
      self.ax_mech.quiver(points[3, 0], points[3, 1], points[3, 2], calc_vel[0], calc_vel[1], calc_vel[2], color='r', label='V Calc')
        
    # Plot velocity components over time
    time = self.times[:frame+1]
    actual_vx = [v[0] for v in self.actual_velocities[:frame+1]]
    actual_vy = [v[1] for v in self.actual_velocities[:frame+1]]
    actual_vz = [v[2] for v in self.actual_velocities[:frame+1]]
    calc_vx = [v[0] for v in self.calculated_velocities[:frame+1]]
    calc_vy = [v[1] for v in self.calculated_velocities[:frame+1]]
    calc_vz = [v[2] for v in self.calculated_velocities[:frame+1]]

    self.vx_actual.set_data(time, actual_vx)
    self.vy_actual.set_data(time, actual_vy)
    self.vz_actual.set_data(time, actual_vz)
    self.vx_calc.set_data(time, calc_vx)
    self.vy_calc.set_data(time, calc_vy)
    self.vz_calc.set_data(time, calc_vz)

    return self.dummyLine,
      
  def animate(self, points_history, times, actual_velocities, calculated_velocities):
    self.points_history = points_history
    self.times = times
    self.actual_velocities = actual_velocities
    self.calculated_velocities = calculated_velocities
    
    anim = FuncAnimation(self.fig, self.update,
                        frames=len(times), 
                        interval=50,
                        blit=True)
    
    plt.tight_layout()
    return HTML(anim.to_jshtml())

In [ ]:
# Initialize the verifier
verifier = Verifier()
points_history = []
times = []
actual_velocities = []
calc_velocities = []

# Time variables
time = 0
dt = 0.02
total_time = 1

# Initial points
angle0, angle1, angle5 = 0, np.pi, np.pi/2
omega0, omega1, omega5 = 0.5, -1, -0.5 
legkinematics.set_motor_angles([angle0, angle1, angle5])
prev_points = legkinematics.get_points()
while time < total_time:
  # Update the angles
  angle0 = (angle0 + omega0 * dt) % (2*np.pi)
  angle1 = (angle1 + omega1 * dt) % (2*np.pi)
  angle5 = (angle5 + omega5 * dt) % (2*np.pi)

  # Forwards kinematics
  legkinematics.set_motor_angles([angle0, angle1, angle5])
  legkinematics.set_motor_omegas([omega0, omega1, omega5])

  # Get results
  points = legkinematics.get_points()
  calc_p3_velocity = legkinematics.get_ee_velocity()
  actual_p3_velocity = (points[3] - prev_points[3]) / dt

  # Store data
  points_history.append(points)
  times.append(time)
  actual_velocities.append(actual_p3_velocity)
  calc_velocities.append(calc_p3_velocity)

  prev_points = points
  time += dt

animation = verifier.animate(points_history, times, actual_velocities, calc_velocities)
display(animation)

## Verify Differential Inverse Kinematics

In [ ]:
# Forward kinematics
motor_angles = [0.1, np.pi, np.pi/2]
omegas = [0.5, -1, -0.5]

legkinematics.set_motor_angles(motor_angles)
legkinematics.set_motor_omegas(omegas)
ee_point = legkinematics.get_ee_point()
velocities = legkinematics.get_ee_velocity()
print("ee_point: ", ee_point)

# Inverse kinematics
legkinematics.set_ee_point(ee_point)
legkinematics.set_ee_velocity(velocities)

motor_angles = legkinematics.get_motor_angles()
omegas = legkinematics.get_motor_omegas()
print(motor_angles)
print(omegas)

In [ ]:
# Initialize the verifier
verifier = Verifier()
legkinematics = LegKinematics(DEBUG=True)
points_history = []
times = []
actual_velocities = []
calc_velocities = []

# Time variables
time = 0
dt = 0.02
total_time = 1

# Initial points
x, y, z = -2, -17, -2
vx, vy, vz = 10, 5, 1

# Init condition
legkinematics.set_ee_point([x, y, z])
legkinematics.set_ee_velocity([vx, vy, vz])
prev_points = legkinematics.get_points()

while time < total_time:
  # Update state
  motor_angles = legkinematics.get_motor_angles()
  omegas = legkinematics.get_motor_omegas()
  angle0 = (motor_angles[0] + omegas[0] * dt) % (2*np.pi)
  angle1 = (motor_angles[1] + omegas[1] * dt) % (2*np.pi)
  angle5 = (motor_angles[2] + omegas[2] * dt) % (2*np.pi)
  legkinematics.set_motor_angles([angle0, angle1, angle5])
  legkinematics.set_ee_velocity([vx, vy, vz]) # Also update omega
  
  # Store results
  points = legkinematics.get_points()
  calc_p3_velocity = legkinematics.get_ee_velocity()
  actual_p3_velocity = (points[3] - prev_points[3]) / dt
  actual_velocities.append(actual_p3_velocity)
  calc_velocities.append(calc_p3_velocity)
  points_history.append(points)
  times.append(time)
  
  prev_points = points
  time += dt

animation = verifier.animate(points_history, times, actual_velocities, calc_velocities)
display(animation)